In [ ]:
# Install esm and other dependencies
! pip install boto3
! pip install esm
! pip install py3Dmol
! pip install matplotlib
! pip install dna-features-viewer

In [1]:
# Uncomment to enable boto3 debug logging.

# import boto3
# import logging
# logging.basicConfig(level=logging.DEBUG)
# boto3.set_stream_logger(name="botocore")

In [ ]:
# Set up AWS credentials using your user profile

import boto3

AWS_REGION='us-east-2'
AWS_PROFILE='partner-prem-dev'

boto3.setup_default_session(region_name=AWS_REGION, profile_name=AWS_PROFILE)

In [ ]:
# Uncomment to set up AWS credentials manually using the shared service user account.

# import os
# import boto3

# AWS_REGION="us-east-2"
# AWS_ACCESS_KEY_ID="foo"
# AWS_SECRET_ACCESS_KEY="bar"

# session = boto3.setup_default_session(
#     aws_access_key_id=AWS_ACCESS_KEY_ID,
#     aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
#     region_name=AWS_REGION
# )

In [11]:
import pprint

sts_client = boto3.client("sts", region_name=AWS_REGION)
identity = sts_client.get_caller_identity()
pprint.pprint(identity)

{'Account': '577638386256',
 'Arn': 'arn:aws:iam::577638386256:user/esm-shared-invoke-dev',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '418',
                                      'content-type': 'text/xml',
                                      'date': 'Mon, 31 Mar 2025 21:44:24 GMT',
                                      'x-amzn-requestid': '4e5a0b1d-7873-4745-9258-4d494ef95cda'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4e5a0b1d-7873-4745-9258-4d494ef95cda',
                      'RetryAttempts': 0},
 'UserId': 'AIDAYM7POJJIM73IM3WFD'}


In [12]:
# CLI output:

# $ terraform output
# sagemaker_endpoints = {
#   "exploratory_model" = {
#     "endpoint_config_name" = "EndpointConfig-Endpoint-ESMC-300M-1-f1eddb86"
#     "endpoint_name" = "Endpoint-ESMC-300M-1-f1eddb86"
#     "endpoint_url" = "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/Endpoint-ESMC-300M-1-f1eddb86/invocations"
#     "forge_model_name" = "esmc-300m-2024-12"
#   }
# }

# Console: https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/endpoints
SAGE_ENDPOINT_NAME = "Endpoint-ESMC-300M-1-2ecf6bf2"

# EvolutionaryScale/Forge Model name (not SageMaker Model Name)
# Update from the Forge Console: https://forge.evolutionaryscale.ai/console
# Options: esm3-open-2024-03, esmc-300m-2024-12, esmc-600m-2024-12
MODEL_NAME = "esmc-300m-2024-12"

In [13]:
from esm.sdk.sagemaker import ESM3SageMakerClient
from esm.sdk.api import ESMProtein, LogitsConfig

sagemaker_client = ESM3SageMakerClient(
   # E.g. "Endpoint-ESMC-6B-1"
   endpoint_name=SAGE_ENDPOINT_NAME,
   # E.g. "esmc-6b-2024-12". Same model names as in Forge.
   model=MODEL_NAME
)

protein = ESMProtein(sequence="AAAAA")
protein_tensor = sagemaker_client.encode(protein)
logits_output = sagemaker_client.logits(
   protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)
print(logits_output.logits, logits_output.embeddings)

ForwardTrackData(sequence=tensor([[-38.0000, -38.0000, -38.0000,  12.6250,  21.6250,  22.3750,  22.0000,
          21.8750,  21.5000,  21.6250,  21.7500,  21.2500,  20.7500,  21.5000,
          21.8750,  20.8750,  20.7500,  20.2500,  20.3750,  20.2500,  21.7500,
          19.8750,  19.8750,  19.3750,  18.3750,   1.3750,  -1.5781,  -3.8750,
         -20.5000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000,
         -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000,
         -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000,
         -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000,
         -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000, -38.0000,
         -38.0000],
        [-40.0000, -40.0000, -40.0000,   5.4688,  20.1250,  20.0000,  18.7500,
          20.6250,  18.8750,  18.3750,  18.6250,  18.5000,  18.2500,  18.0000,
          18.7500,  17.7500,  17.5000,  17.1250,  17.7500,  16.8750,  22.1250,
      